In [1]:
import human_eval
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import os
import numpy as np
import scipy as sp
# from awq import AutoAWQForCausalLM
from human_eval.data import write_jsonl, read_problems
import torch
from human_eval.data import write_jsonl, read_problems
import time 

In [4]:
# load model and tokenizer
model_path = "Microsoft/phi-1"
# tokenizer_path = 'WizardLM/WizardCoder-1B-V1.0'  
# model_path = "/home/justin/.cache/huggingface/hub/models--microsoft--phi-1_5"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True).to("cuda:0").eval()



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


A new version of the following files was downloaded from https://huggingface.co/Microsoft/phi-1:
- configuration_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/Microsoft/phi-1:
- modeling_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [15]:
prompt = '''
def unique(l: list):
    """Return sorted unique elements in a list
    >>> unique([5, 3, 5, 2, 3, 3, 9, 0, 123])
    [0, 2, 3, 5, 9, 123]
    """
'''

In [13]:
def generate_one_completion(prompt):
    inputs = tokenizer(prompt.rstrip(), return_tensors="pt").to("cuda")
    # start time
    start_time = time.time()
    generation_output = model.generate(
        **inputs,
        max_new_tokens=1028,
        eos_token_id=tokenizer.eos_token_id,
        return_dict_in_generate=True,
    )

    s = generation_output.sequences[0]
    output = tokenizer.decode(s, skip_special_tokens=True)
    # end time
    end_time = time.time()
    # calculate time taken
    time_taken = end_time - start_time
    # print("time taken: ", time_taken)
    # calcuate # of tokens
    num_tokens = len(output)
    # tokens per second
    tokens_per_second = num_tokens / time_taken
    print("tokens per second: ", tokens_per_second)
    return output

# print(output)

In [16]:
output = generate_one_completion(prompt)
print(output)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokens per second:  768.660930140341

def unique(l: list):    """Return sorted unique elements in a list    >>> unique([5, 3, 5, 2, 3, 3, 9, 0, 123])    [0, 2, 3, 5, 9, 123]    """
    return sorted(set(l))




In [17]:

message = "print hello world in python and javascript"



In [18]:
#tokenize 

inputs = tokenizer(message, return_tensors="pt").to(model.device)

# tokens = tokenizer.tokenize(message)
# input_ids = tokenizer.encode(message, return_tensors="pt")
# print(tokens)
print (inputs)
# print(input_ids)

{'input_ids': tensor([[ 4798, 23748,   995,   287, 21015,   290, 44575]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [19]:
with torch.no_grad():
    outputs = model.generate(**inputs, max_length=50)

In [21]:
# print(outputs.last_hidden_state.shape)
#convert logits to tokens via softmax

output_tokens =tokenizer.decode(outputs[0])
# print(outputs.logits)
# logits = sp.special.softmax (outputs.logits[0].detach().numpy())
# print(logits)

# print(logits.shape)
# ouput_id = []
# for i in logits:
#     print(max(i))
#     ouput_id.append(np.argmax(i))

# # print(outputs)
# ouput_tokens = tokenizer.convert_ids_to_tokens(ouput_id)
                                               
print(output_tokens)

print hello world in python and javascript"
    """
    words = s.split()
    for i in range(len(words)):
        if words[i].lower() == "hello":
            words[i]


In [22]:
problems = read_problems()

In [23]:
num_samples_per_task = 1
samples = [
    dict(task_id=task_id, completion=generate_one_completion(problems[task_id]["prompt"]))
    for task_id in problems
    for _ in range(num_samples_per_task)
]
write_jsonl("samples.jsonl", samples)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokens per second:  473.7105353291949


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokens per second:  468.9132565268968


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokens per second:  632.9785048662314


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokens per second:  447.48200191850447


KeyboardInterrupt: 

In [24]:
problems['HumanEval/0']

{'task_id': 'HumanEval/0',
 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n',
 'entry_point': 'has_close_elements',
 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n',
 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9,

In [41]:
from typing import Iterable, Dict
import json
HUMAN_EVAL = 'samples.jsonl'
def read_problems(evalset_file: str = HUMAN_EVAL) -> Dict[str, Dict]:
    return {task["task_id"]: task for task in stream_jsonl(evalset_file)}


In [38]:
def stream_jsonl(filename: str) -> Iterable[Dict]:
    """
    Parses each jsonl line and yields it as a dictionary
    """
    if filename.endswith(".gz"):
        with open(filename, "rb") as gzfp:
            with gzip.open(gzfp, 'rt') as fp:
                for line in fp:
                    if any(not x.isspace() for x in line):
                        yield json.loads(line)
    else:
        with open(filename, "r") as fp:
            for line in fp:
                if any(not x.isspace() for x in line):
                    yield json.loads(line)


In [42]:
# evaluate sample performance 
#read samples from json

samples_eval = read_problems()
sample_path = os.path.join(os.getcwd(), "completions/")
for sample in samples:
    sample_file = os.path.join(sample_path, f"{sample['task_id']}.py")
    #write completion to file 
    with open(sample_file, "w") as f:
        f.write(sample['completion'])
    # print(sample['completion'])    
